In [2]:
import sentencepiece as spm
from gensim.models import Word2Vec, KeyedVectors
import numpy as np

Tokenzier

In [3]:
#설정값
txtPath = "tokenized_data.txt"
tokenizerModelName = "token" #만들 .model,.vocab 파일 이름
tokenizerPath = "./" + tokenizerModelName + ".model"
vocabSize = 24000 #최대 token의 수 제한. 만들 수 있는 token의 수보다 크게 잡을 경우 에러 뜸

In [4]:
def makeTokenizerModel(txtPath, modelName, vocabSize):
    spm.SentencePieceTrainer.Train(
        f"--input={txtPath} --model_prefix={modelName} --vocab_size={vocabSize + 7}" + 
        " --model_type=bpe" +
        " --max_sentence_length=999999" + # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
        " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
        " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
        " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
        " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [5]:
makeTokenizerModel(txtPath, tokenizerModelName, vocabSize) 
print("tokenizer 모델 생성 완료")

tokenizer 모델 생성 완료


In [6]:
tokenizer = spm.SentencePieceProcessor()
tokenizer.Load(tokenizerPath)
textForTest = "부터 까지의 자연수가 각각 하나씩 적힌 장의 카드에서 임의로 장의 카드를 뽑을 때, 가장 작은 수가  일 확률은?"
print(tokenizer.encode_as_pieces(textForTest))

['▁\ue034', '부터', '▁\ue034\ue03d', '까지의', '▁자연수가', '▁각각', '▁하나씩', '▁적힌', '▁\ue034\ue03d', '장의', '▁카드에서', '▁임의로', '▁\ue036', '장의', '▁카드를', '▁뽑을', '▁때', ',', '▁가장', '▁작은', '▁수가', '▁\ue037', '▁일', '▁확률은', '?']


Word embedder

In [7]:
#설정값
embedderModelName = "embedder.model"
embeddingSize = 32

In [8]:
def makeEmbedderModel(tokenizer, txtPath, modelName, embeddingSize):
    tokenizedQuestions = []
    with open(txtPath, "rt", encoding="UTF8") as txtFile:
        txtFileLines = txtFile.readlines()
        for question in txtFileLines:
            tokenizedResult = tokenizer.encode_as_pieces(question)
            if len(tokenizedResult) > 1:
                tokenizedQuestions.append(tokenizedResult)
    
    model = Word2Vec(sentences=tokenizedQuestions, size=embeddingSize, window=5, min_count=5, workers=4, sg=1)
    model.save(modelName)

In [9]:
makeEmbedderModel(tokenizer, txtPath, embedderModelName, embeddingSize)
print("embedder 모델 생성 완료")


embedder 모델 생성 완료


문장 embedding 및 크기 맞추기

In [10]:
def embedQuestion(tokenizer, wordEmbedder, question):
    tokensInQuestion = tokenizer.encode_as_pieces(question)
    embeddedResult = []
    for token in tokensInQuestion:
        try:
            embeddedVector = wordEmbedder.wv.get_vector(token)
            embeddedResult.append(embeddedVector)
        except:
            pass
    return np.array(embeddedResult)

def embedEntireQuestion(tokenizer, wordEmbedder, txtPath):
    embedResult = []
    with open(txtPath, "rt", encoding="UTF8") as txtFile:
        txtFileLines = txtFile.readlines()
        for question in txtFileLines:
            if len(question) > 1:
                embedResult.append(embedQuestion(tokenizer, wordEmbedder, question))
    return embedResult

def getMaxEmbedLength(embedResult):
    currentMax = -1
    for question in embedResult:
        if question.shape[0] > currentMax:
            currentMax = question.shape[0]
    return currentMax

def fitVector(embedResult, embeddingSize, maxLength):
    for i in range(len(embedResult)):
        zeroArrayTostack = np.zeros([maxLength - embedResult[i].shape[0], embeddingSize])
        embedResult[i] = np.vstack([embedResult[i], zeroArrayTostack])

def embedAndFitQuestion(tokenizer, wordEmbedder, question, embeddingSize, lengthToFit):
    embeddedQuestion = embedQuestion(tokenizer, wordEmbedder, question)
    zeroArrayTostack = np.zeros([lengthToFit - embeddedQuestion.shape[0], embeddingSize])
    return np.vstack([embeddedQuestion, zeroArrayTostack])

In [11]:
#모든 문제를 읽어서 tokenize 및 word embedding 한 뒤, 최대 크기에 맞춰서 0으로 채움
wordEmbedder = Word2Vec.load(embedderModelName)

result = embedEntireQuestion(tokenizer, wordEmbedder, txtPath)
print("최장 길이 문제의 토큰 수:", getMaxEmbedLength(result))
fitVector(result, embeddingSize, 160)


최장 길이 문제의 토큰 수: 154


In [12]:
question = "미분가능한 함수 에 대하여       일 때 상수 ,  에 대하여 의 값을 구하여라."
question = embedAndFitQuestion(tokenizer, wordEmbedder, question, embeddingSize, 160)
print(question.shape)
print(question)

(160, 32)
[[ 0.11570789 -0.6297195  -0.06389525 ... -0.83909971 -0.90536386
  -0.57258922]
 [-0.18262972 -0.33033806  0.04562072 ... -0.37462974 -1.0022285
  -0.70879304]
 [-0.03199902 -0.36972684 -0.00715211 ... -0.48808759 -0.82278657
  -0.53067869]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [37]:
import json
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from torchsummary import summary as summary_

In [14]:
learning_rate = 0.001
num_epochs = 10
validation_split = 0.2
shuffle_dataset = True
batch_size = 64
random_seed = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
class customDataset(data.Dataset):
    def __init__(self, x_tensor, y_tensor):
        super(customDataset, self).__init__()
        
        self.x = x_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)


In [16]:
filePath = 'mixedQuestion.txt'
def bringData(file):
    qList = []
    yList = []
    with open(file, 'rt', encoding="UTF8") as questionFile:
        questions = json.load(questionFile)
    for q in questions:
        embedded = embedAndFitQuestion(tokenizer, wordEmbedder, q['question'], embeddingSize, 160)
        channeled = [embedded]
        channeled = np.array(channeled)
        qList.append(channeled)
        parse = q['value']
        parse = int(parse.find('1')/3)
        #parse = [int(parse[1]), int(parse[4]),int(parse[7]),int(parse[10]),int(parse[13]),int(parse[16])]
        yList.append(parse)
    return qList, yList

In [17]:
x_data, y_data = bringData(filePath)

In [18]:
x_data = torch.FloatTensor(x_data)
y_data = torch.LongTensor(y_data)

In [19]:
dataset = customDataset(x_data, y_data)

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]    
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size = batch_size, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size = batch_size, sampler=valid_sampler)

In [20]:
class questionNet(nn.Module):
    def __init__(self):
        super(questionNet, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1,16,(3,32),padding=0), nn.ReLU(),
            nn.Conv2d(16,16,(3,1), padding=0),nn.ReLU(), #16 80 1
            nn.Conv2d(16,64,(3,1), padding=0),nn.ReLU(), #16 80 1
            nn.MaxPool2d(1,2), #16 40 1
            nn.BatchNorm2d(64),

            nn.Conv2d(64,128,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.Conv2d(128,128,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.MaxPool2d(1,2),                                    #64 20 1
            nn.BatchNorm2d(128),

            nn.Conv2d(128,256,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.Conv2d(256,256,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.MaxPool2d(1,2),                                    #64 20 1
            nn.BatchNorm2d(256),

            nn.Conv2d(256,512,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.Conv2d(512,512,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.MaxPool2d(1,2),                                    #64 20 1
            nn.BatchNorm2d(512),

            nn.Conv2d(512,512,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.Conv2d(512,512,(3,1), padding=0),nn.ReLU(),#64 40 1
            nn.MaxPool2d(1,2),                                    #64 20 1
            nn.BatchNorm2d(512),

            nn.Flatten(),

            nn.Linear(1024, 256), nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 64), nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64,6)
        )
        
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.conv(x)
        return x.view([-1,6])

    def inferrence(self, x):
        x = self.forward(x)
        x = self.softmax(x)
        return x

qModel = questionNet()

In [39]:
summary_(qModel,(1,160,32),batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [64, 16, 158, 1]           1,552
              ReLU-2           [64, 16, 158, 1]               0
            Conv2d-3           [64, 16, 156, 1]             784
              ReLU-4           [64, 16, 156, 1]               0
            Conv2d-5           [64, 64, 154, 1]           3,136
              ReLU-6           [64, 64, 154, 1]               0
         MaxPool2d-7            [64, 64, 77, 1]               0
       BatchNorm2d-8            [64, 64, 77, 1]             128
            Conv2d-9           [64, 128, 75, 1]          24,704
             ReLU-10           [64, 128, 75, 1]               0
           Conv2d-11           [64, 128, 73, 1]          49,280
             ReLU-12           [64, 128, 73, 1]               0
        MaxPool2d-13           [64, 128, 37, 1]               0
      BatchNorm2d-14           [64, 128

In [21]:
def train_model(model, num_epochs=1, lr = learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        total = 0
        correct = 0
        print('\nEpoch: %d' % (epoch+1))
        for batch_idx, data in enumerate(train_loader):
            image, label = data
            # Grad initialization
            optimizer.zero_grad()
            # Forward propagation
            output = model(image)
            # Calculate loss
            
            #print(Variable(index).type(torch.LongTensor))
            loss = criterion(output, label)
            # Backprop
            loss.backward()
            # Weight update
            optimizer.step()

            train_loss += loss.item()
            _,predicted = output.max(1)
            total += label.size(0)
            correct += predicted.eq(label).sum().item()

            if (batch_idx+1) % 100 == 0:
                print("Step: {}/{} | train_loss: {:.4f} | Acc:{:.3f}%".format(batch_idx+1, len(train_loader), train_loss/1000, 100.*correct/total), "-----> ", end = "")
                test_model(model)

def test_model(model):
    test_loss=0
    correct=0
    total=0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            question, label = data
            output = model(question)
            loss = criterion(output,label)
            
            test_loss += loss.item()
            _,predicted = output.max(1)
            total += label.size(0)
            correct += predicted.eq(label).sum().item()
            
            if(batch_idx+1) == len(test_loader):
                print("Test_loss: {:.4f} | Test_Acc:{:.3f}%".format(test_loss/1000, 100.*correct/total))                

In [22]:
train_model(qModel, num_epochs = 10)


Epoch: 1
Step: 100/731 | train_loss: 0.1718 | Acc:26.609% -----> Test_loss: 0.2761 | Test_Acc:36.500%
Step: 200/731 | train_loss: 0.3104 | Acc:34.469% -----> Test_loss: 0.2364 | Test_Acc:47.625%
Step: 300/731 | train_loss: 0.4323 | Acc:39.964% -----> Test_loss: 0.2083 | Test_Acc:54.882%
Step: 400/731 | train_loss: 0.5447 | Acc:43.926% -----> Test_loss: 0.1934 | Test_Acc:58.391%
Step: 500/731 | train_loss: 0.6452 | Acc:47.378% -----> Test_loss: 0.1775 | Test_Acc:62.208%
Step: 600/731 | train_loss: 0.7414 | Acc:49.901% -----> Test_loss: 0.1701 | Test_Acc:63.714%
Step: 700/731 | train_loss: 0.8332 | Acc:52.013% -----> Test_loss: 0.1600 | Test_Acc:65.691%

Epoch: 2
Step: 100/731 | train_loss: 0.0869 | Acc:67.641% -----> Test_loss: 0.1653 | Test_Acc:64.228%
Step: 200/731 | train_loss: 0.1729 | Acc:67.867% -----> Test_loss: 0.1504 | Test_Acc:69.491%
Step: 300/731 | train_loss: 0.2565 | Acc:68.484% -----> Test_loss: 0.1478 | Test_Acc:69.833%
Step: 400/731 | train_loss: 0.3358 | Acc:68.992% -

model save and load

In [67]:
test_model(qModel)

Test_loss: 0.0908 | Test_Acc:84.082%


In [23]:
torch.save(qModel.state_dict(), "./trainedModel")

In [24]:
model = questionNet()
model.load_state_dict(torch.load("./trainedModel"))

<All keys matched successfully>

question type inferrence using model

In [65]:
def indexToString(modelOutput):
    if modelOutput == 0:
        return "기하와벡터"
    elif modelOutput == 1:
        return "미적분1"
    elif modelOutput == 2:
        return "미적분2"
    elif modelOutput == 3:
        return "수학1"
    elif modelOutput == 4:
        return "수학2"
    elif modelOutput == 5:
        return "확률과통계"

def getTypeOfQuestion(question, model):
    model.eval()
    questionToTensor = torch.Tensor(embedAndFitQuestion(tokenizer, wordEmbedder, question, embeddingSize, 160))
    modelOutput = model.inferrence(questionToTensor.view([1,1,160,32]))
    return indexToString(modelOutput.argmax().item()), modelOutput[0][modelOutput.argmax().item()].item() * 100

In [66]:
questionList = list()
#2021 수능 수학 가형 4번
questionList.append("두 사건  ,  에 대하여 P, P, PP 일 때, P∩의 값은?")

#2021 수능 수학 가형 6번
questionList.append("정규분포 N 을 따르는 모집단에서 크기가 인 표본을 임의추출하여 구한 표본평균을  라 할 때, E  의 값은?")

#2021 수능 수학 가형 8번
questionList.append("곡선    과  축 및 두 직선   ln  ,   ln  로 둘러싸인 부분의 넓이는?")

#2021 수능 수학 가형 21번
questionList.append("수열 은  이고, 모든 자연수 에 대하여 다음 조건을 만족시킨다. ( 가 )       ×     (나)  × 일때,  의값은?")

#2021 수능 수학 가형 30번
questionList.append("최고차항의 계수가 인 삼차함수 에 대하여 실수 전체의 집합에서 정의된 함수 sin가 다음 조건을 만족시킨다. (가) 에서 함수 가 극대가 되는 의 개수가 이고, 이때 극댓값이 모두 동일하다. (나) 함수 의 최댓값은  이고 최솟값은 이다.           일 때 ,      의 값 을 구 하 시 오 . ( 단 ,  와  는 유리수이다.)")

#2021 수능 수학 나형 29번
questionList.append("숫자 , , , , 가 하나씩 적힌 개의 공이 들어 있는 주머니가 있다. 이 주머니와 한 개의 주사위를 사용하여 다음 규칙에 따라 점수를 얻는 시행을 한다. 주머니에서 임의로 한 개의 공을 꺼내어 꺼낸 공에 적힌 수가 이면 주사위를 번 던져서 나오는 세 눈의 수의 합을 점수로 하고, 꺼낸 공에 적힌 수가 이면 주사위를 번 던져서 나오는 네 눈의 수의 합을 점수로 한다. 이시행을한번하여얻은점수가점일확률은 이다.   의 값을 구하시오. (단, 와 는 서로소인 자연수이다.)")

#2021 수능 수학 나형 30번
questionList.append("함수 는 최고차항의 계수가 인 삼차함수이고, 함수 는 일차함수이다. 함수 를               ≥ 이라 하자. 함수 가 실수 전체의 집합에서 미분가능하고, , 일 때, 의 값을 구하시오.")

for question in questionList:
    classificationResult = getTypeOfQuestion(question, model)
    print("question: ", question)
    print("분류 결과: ", classificationResult[0], ", 신뢰도: ", classificationResult[1], "%")
    print("")



question:  두 사건  ,  에 대하여 P, P, PP 일 때, P∩의 값은?
분류 결과:  확률과통계 , 신뢰도:  90.9182608127594 %

question:  정규분포 N 을 따르는 모집단에서 크기가 인 표본을 임의추출하여 구한 표본평균을  라 할 때, E  의 값은?
분류 결과:  확률과통계 , 신뢰도:  99.66621398925781 %

question:  곡선    과  축 및 두 직선   ln  ,   ln  로 둘러싸인 부분의 넓이는?
분류 결과:  미적분2 , 신뢰도:  99.75231289863586 %

question:  수열 은  이고, 모든 자연수 에 대하여 다음 조건을 만족시킨다. ( 가 )       ×     (나)  × 일때,  의값은?
분류 결과:  수학2 , 신뢰도:  89.8624062538147 %

question:  최고차항의 계수가 인 삼차함수 에 대하여 실수 전체의 집합에서 정의된 함수 sin가 다음 조건을 만족시킨다. (가) 에서 함수 가 극대가 되는 의 개수가 이고, 이때 극댓값이 모두 동일하다. (나) 함수 의 최댓값은  이고 최솟값은 이다.           일 때 ,      의 값 을 구 하 시 오 . ( 단 ,  와  는 유리수이다.)
분류 결과:  미적분2 , 신뢰도:  84.33175086975098 %

question:  숫자 , , , , 가 하나씩 적힌 개의 공이 들어 있는 주머니가 있다. 이 주머니와 한 개의 주사위를 사용하여 다음 규칙에 따라 점수를 얻는 시행을 한다. 주머니에서 임의로 한 개의 공을 꺼내어 꺼낸 공에 적힌 수가 이면 주사위를 번 던져서 나오는 세 눈의 수의